In [ ]:
from databricks.connect import DatabricksSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    DecimalType,
    TimestampType,
)
from pyspark.sql.functions import create_map, lit

# Wouldn't need this for the job runtime as it defaults to use SparkSession already
# spark = DatabricksSession.builder.serverless().profile("fe_kfs").getOrCreate()

In [ ]:
# To retrieve the parameters at the job run time for metadata

pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

In [ ]:
custom_schema = StructType(
    [
        StructField("ride_id", StringType(), True),
        StructField("rideable_type", StringType(), True),
        StructField("started_at", TimestampType(), True),
        StructField("ended_at", TimestampType(), True),
        StructField("start_station_name", StringType(), True),
        StructField("start_station_id", StringType(), True),
        StructField("end_station_name", StringType(), True),
        StructField("end_station_id", StringType(), True),
        StructField("start_lat", DecimalType(), True),
        StructField("start_lng", DecimalType(), True),
        StructField("end_lat", DecimalType(), True),
        StructField("end_lng", DecimalType(), True),
        StructField("member_casual", StringType(), True),
    ]
)

In [ ]:
df = (
    spark.read.format("csv")
    .options(header=True)
    .schema(custom_schema)
    .load(
        f"/Volumes/{catalog}/00_landing/source_citibike_data/JC-202503-citibike-tripdata.csv"
    )
)

In [ ]:
df = df.withColumn(
    "metadata",
    create_map(
        lit("pipeline_id"),
        lit(pipeline_id),
        lit("run_id"),
        lit(run_id),
        lit("task_id"),
        lit(task_id),
        lit("processed_timestamp"),
        lit(processed_timestamp),
    ),
)

In [ ]:
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(
    f"{catalog}.01_bronze.jc_citibike"
)